In [32]:
import pandas as pd
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [33]:
medium_data = pd.read_csv('merged_data.csv')
medium_data.head()

C:\Users\mushf\AppData\Local\Temp\ipykernel_8764\1537209427.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  medium_data = pd.read_csv('merged_data.csv')


,news_title,news_description,country,date
0,প্রধানমন্ত্রীর ৭৫তম জন্মদিন উপলক্ষ্যে জাতীয় বা...,প্রধানমন্ত্রীর ৭৫তম জন্মদিন উপলক্ষ্যে জাতীয় বা...,all_country,NaN
1,শীতের আগেই ত্বকের যত্ন নিবেন যেভাবে,শীতের আগেই ত্বকের যত্ন নিবেন যেভাবেসামনে শীত। ...,all_country,NaN
2,প্রেমের সম্পর্ক গোপন রাখার ৫টি জাদুকরি উপায়,প্রেমের সম্পর্ক গোপন রাখার ৫টি জাদুকরি উপায় সম...,all_country,NaN
3,ফ্রন্ট লোড ওয়াশিং মেশিন উন্নত লাইফস্টাইলে উদ্...,ফ্রন্ট লোড ওয়াশিং মেশিন উন্নত লাইফস্টাইলে উদ্...,all_country,NaN
4,শীতে রোগবালাই থেকে মুক্ত থাকবেন যেভাবে,শীতে রোগবালাই থেকে মুক্ত থাকবেন যেভাবেশীত হয়তো...,all_country,NaN


In [34]:
print("Number of records: ", medium_data.shape[0])
print("Number of fields: ", medium_data.shape[1])

Number of records:  263798
Number of fields:  4


In [35]:
medium_data['news_description']

0         প্রধানমন্ত্রীর ৭৫তম জন্মদিন উপলক্ষ্যে জাতীয় বা...
1         শীতের আগেই ত্বকের যত্ন নিবেন যেভাবেসামনে শীত। ...
2         প্রেমের সম্পর্ক গোপন রাখার ৫টি জাদুকরি উপায় সম...
3         ফ্রন্ট লোড ওয়াশিং মেশিন উন্নত লাইফস্টাইলে উদ্...
4         শীতে রোগবালাই থেকে মুক্ত থাকবেন যেভাবেশীত হয়তো...
                                ...                        
263793    রাশিয়ার প্রেসিডেন্ট ভ্লাদিমির পুতিন ও চীনা প্র...
263794    সিরিয়ায় ইদলিব প্রদেশে বিদ্রোহীদের অধিকৃত একটি ...
263795    সিরিয়ার পার্শ্ববর্তী অঞ্চলে আইএসের বিরুদ্ধে আব...
263796    ইরাকের বাগদাদে একটি শিয়া মসজিদে পৃথক বোমা হামল...
263797                                                  NaN
Name: news_description, Length: 263798, dtype: object

In [36]:
medium_data['news_description'] = medium_data['news_description'].apply(lambda x: str(x).replace(u'\xa0',' '))
medium_data['news_description'] = medium_data['news_description'].apply(lambda x: str(x).replace('\u200a',' '))

In [37]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
tokenizer.fit_on_texts(medium_data['news_description'])
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)

Total number of words:  559471


In [38]:
input_sequences = []

# Iterate over the first 100 rows of the 'title' column
for line in medium_data['news_description'][:100]: #[:100]
    token_list = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print("Total input sequences: ", len(input_sequences))

Total input sequences:  11550


In [39]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [42]:
# create features and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = labels


In [43]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=50, verbose=1)
#print model.summary()
print(model)

Epoch 1/50


ValueError: in user code:

    File "c:\Users\mushf\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\mushf\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\mushf\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\mushf\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\mushf\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "c:\Users\mushf\anaconda3\Lib\site-packages\keras\src\engine\compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, samp